In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def neighbours(i,j):
    return [[i+1,j],[i-1,j],[i,j+1],[i,j-1]]

def outlines_target(masque):
    outlines = []
    for i in range(len(masque)):
        for j in range(len(masque[0])):
            if masque[i][j] == 1:
                for n in neighbours(i,j):
                    if masque[n[0]][n[1]] == 0 and n not in outlines:
                        outlines.append(n)
    return np.array(outlines)


In [2]:
masque = np.array([[0,0,0,0,0],
                    [0,0,0,0,0],
                    [0,0,1,1,0],
                    [0,0,0,0,0],
                    [0,0,0,0,0]])

outline= outlines_target(masque)
print(outline)
outline_in_array = np.zeros(masque.shape)
outline_in_array[outline[:,0], outline[:,1]] = 1
plt.imshow(outline_in_array, cmap='gray')
plt.show()

TypeError: outlines_target() missing 1 required positional argument: 'zone'